#  1. Pretraining

## 1.1 Load and Down Sample data

In [3]:
import pandas as pd

# load MIMIC-III
# mimic3 = pd.read_csv('/content/drive/MyDrive/uiuc/NOTEEVENTS.csv.gz', compression='gzip')

# load MIMIC-IV discharge summaries and rad reports
# mimic4_discharge = pd.read_csv('/content/drive/MyDrive/uiuc/discharge.csv.gz', compression='gzip')
mimic4_radiology = pd.read_csv('/content/drive/MyDrive/uiuc/radiology.csv.gz', compression='gzip')



In [4]:
mimic4_radiology.head()

,note_id,subject_id,hadm_id,note_type,note_seq,charttime,storetime,text
0,10000032-RR-14,10000032,22595853.0,RR,14,2180-05-06 21:19:00,2180-05-06 23:32:00,EXAMINATION: CHEST (PA AND LAT)\n\nINDICATION...
1,10000032-RR-15,10000032,22595853.0,RR,15,2180-05-06 23:00:00,2180-05-06 23:26:00,EXAMINATION: LIVER OR GALLBLADDER US (SINGLE ...
2,10000032-RR-16,10000032,22595853.0,RR,16,2180-05-07 09:55:00,2180-05-07 11:15:00,"INDICATION: ___ HCV cirrhosis c/b ascites, hi..."
3,10000032-RR-18,10000032,NaN,RR,18,2180-06-03 12:46:00,2180-06-03 14:01:00,EXAMINATION: Ultrasound-guided paracentesis.\...
4,10000032-RR-20,10000032,NaN,RR,20,2180-07-08 13:18:00,2180-07-08 14:15:00,EXAMINATION: Paracentesis\n\nINDICATION: ___...


In [5]:
mimic4_radiology.shape

(2321355, 8)

In [6]:
# prompt: randomly keep only 5 percent of the data

import numpy as np

# Sample 5% of the data
sampled_df = mimic4_radiology.sample(40000, random_state=42) # Set random_state for reproducibility

print(sampled_df.shape)
sampled_df.head()


(40000, 8)


,note_id,subject_id,hadm_id,note_type,note_seq,charttime,storetime,text
485326,12105403-RR-51,12105403,NaN,RR,51,2201-10-22 15:04:00,2201-10-23 08:21:00,FLUOROSCOPIC STEROID INJECTION INTO THE LEFT K...
1927556,18301101-RR-35,18301101,26543664.0,RR,35,2126-09-26 09:05:00,2126-09-26 09:57:00,EXAMINATION: PORTABLE HEAD CT W/O CONTRAST Q1...
910696,13935134-RR-30,13935134,21712098.0,RR,30,2176-04-10 18:00:00,2176-04-10 20:35:00,EXAMINATION: CHEST (PORTABLE AP)\n\nINDICATIO...
52091,10232271-RR-4,10232271,24699170.0,RR,4,2133-05-14 13:55:00,2133-05-15 16:40:00,CLINICAL HISTORY ___ year old woman with know...
2301238,19913826-RR-22,19913826,NaN,RR,22,2159-03-17 20:51:00,2159-03-17 21:40:00,"EXAMINATION: PELVIS U.S., TRANSVAGINAL\n\nIND..."


In [7]:
# Saving the sampled data so it can be reused later
sampled_df.to_csv('/content/drive/MyDrive/uiuc/sampled_df.csv', index=False)

## 1.2 Set up transformers Dataset

In [ ]:
!pip install datasets

In [8]:
sampled_df = pd.read_csv('/content/drive/MyDrive/uiuc/sampled_df.csv')

In [9]:
# prompt: create train and test on sampled_df

from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
train_df, test_df = train_test_split(sampled_df, test_size=0.2, random_state=42)

print("Training set shape:", train_df.shape)
print("Testing set shape:", test_df.shape)


Training set shape: (32000, 8)
Testing set shape: (8000, 8)


In [ ]:
!pip install medspacy

In [10]:
import medspacy
nlp = medspacy.load()

def detect_entity_spans(text: str):
    """
    Returns a list of (char_start, char_end) for each clinical entity in `text`.
    """
    doc = nlp(text)
    return [(ent.start_char, ent.end_char) for ent in doc.ents]

# Example
text = "Patient presents with chest pain and was prescribed aspirin."
print(detect_entity_spans(text))
# → [(18, 28), (48, 55)]

from transformers import T5TokenizerFast

tokenizer = T5TokenizerFast.from_pretrained("t5-base")

def add_token_spans(example):
    """
    Adds a list of token_index spans for each example,
    by mapping char-level clinical spans to token indices.
    """
    encoding = tokenizer(
        example["text"],
        return_offsets_mapping=True,
        truncation=True,
        max_length=512,
    )
    example["input_ids"] = encoding["input_ids"]
    offsets = encoding["offset_mapping"]
    entity_spans = detect_entity_spans(example["text"])
    token_spans = []
    for start_char, end_char in entity_spans:
        # find the first token whose span overlaps start_char,
        # and the last token whose span overlaps end_char
        token_start = next(i for i,(s,e) in enumerate(offsets) if s <= start_char < e)
        token_end   = next(i for i,(s,e) in enumerate(offsets) if s < end_char <= e)
        token_spans.append((token_start, token_end))
    example["entity_token_spans"] = token_spans
    return example



[]


In [11]:
import pandas as pd
from datasets import Dataset
from transformers import (
    T5TokenizerFast,
    T5Config,
    T5ForConditionalGeneration,
    DataCollatorForLanguageModeling,
    TrainingArguments,
    Trainer,
)


max_length = 256  # adjust as needed

train_dataset = Dataset.from_pandas(train_df[['text']])
eval_dataset = Dataset.from_pandas(test_df[['text']])



In [ ]:
# prompt: create a t5-base tokenizer and add in the special DEID tokens, which are the values in the DEID dictionary

from transformers import T5Tokenizer

tokenizer = T5TokenizerFast.from_pretrained("t5-base")


def tokenize_for_pretraining(examples):
    # truncate/pad to max_length; you can adjust max_length to GPU memory
    tokens = tokenizer(
        examples["text"],
        max_length=max_length,
        truncation=True,
        padding="max_length",
    )
    # for span-masking, labels = original input_ids
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

tokenized_ds_train = train_dataset.map(
    add_token_spans,
    batched=True,
    remove_columns=["text"],
)

tokenized_ds_eval = eval_dataset.map(
    add_token_spans,
    batched=True,
    remove_columns=["text"],
)


## 1.3 Pretraining T5-base on sampled MIMIC4 Rad reports

In [10]:
# Initialize model
config    = T5Config.from_pretrained("t5-base")
model     = T5ForConditionalGeneration(config)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,   # ~span density; you can tune this
)

In [11]:
import os
os.environ["WANDB_DISABLED"] = "true"

# 6. Training arguments
training_args = TrainingArguments(
    output_dir="t5_clinical_pretrain",
    overwrite_output_dir=True,
    num_train_epochs=3,
    eval_strategy="steps",
    eval_steps=200,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    learning_rate=1e-4,
    warmup_steps=100,
    logging_steps=200,
    push_to_hub=False,
    fp16=False,
)

# 10. Instantiate the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds_train,
    eval_dataset=tokenized_ds_eval,
    data_collator=data_collator,
)

# 11. Launch training
trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss
200,6.272600,4.286455
400,4.070900,3.536233
600,3.482800,3.123433
800,3.182900,2.859792
1000,2.930600,2.651595
1200,2.714700,2.492981
1400,2.566400,2.360949
1600,2.469800,2.266171
1800,2.363700,2.174942
2000,2.257700,2.104993


TrainOutput(global_step=3000, training_loss=2.8521390075683595, metrics={'train_runtime': 3994.8626, 'train_samples_per_second': 24.031, 'train_steps_per_second': 0.751, 'total_flos': 2.922997874688e+16, 'train_loss': 2.8521390075683595, 'epoch': 3.0})

In [12]:
trainer.save_model("/content/drive/MyDrive/uiuc/t5_base_mimic")


# 2. Finetune on Downsteam Tasks - RadQA

## 2.1 Set up data loading, Training and eval functions

In [ ]:
!pip install evaluate

In [11]:
import json
from pathlib import Path
import os
os.environ["WANDB_DISABLED"] = "true"
import evaluate
from datasets import Dataset
from transformers import (
    T5ForConditionalGeneration,
    T5TokenizerFast,
    DataCollatorForSeq2Seq,
    TrainingArguments,
    Trainer,
)

def load_radqa(path: str):
    """Load a RadQA-format JSON and flatten to a list of QA dicts."""
    items = []
    data = json.loads(Path(path).read_text())
    for article in data["data"]:
        for para in article["paragraphs"]:
            context = para["context"]
            for qa in para["qas"]:
                items.append({
                    "id":            qa["id"],
                    "question":      qa["question"],
                    "context":       context,
                    "answers":       qa["answers"],
                    "is_impossible": qa["is_impossible"],
                })
    return items

def train_model(
    base_model_path: str,
    save_model_path: str,
    train_file: str,
    dev_file: str,
    prefix: str = "question: ",
    num_epochs: int = 3,
    n: int = None,
):
    """
    Fine-tune a T5 model on RadQA.
    If n is set, randomly sample n examples from the train split.
    """
    # 1. Load model & tokenizer
    model     = T5ForConditionalGeneration.from_pretrained(base_model_path)
    tokenizer = T5TokenizerFast.from_pretrained("t5-base")

    # 2. Prepare raw datasets
    raw_train = Dataset.from_list(load_radqa(train_file))
    raw_dev   = Dataset.from_list(load_radqa(dev_file))

    # 2a. Sample n examples if requested
    if n is not None:
        raw_train = raw_train.shuffle(seed=42).select(range(n))

    # 3. Preprocessing fn
    def preprocess_fn(examples):
        inputs = [
            prefix + q + "  context: " + c
            for q, c in zip(examples["question"], examples["context"])
        ]
        model_inputs = tokenizer(
            inputs, max_length=1024, truncation=True, padding="max_length"
        )
        targets = [
            ans_list[0]["text"] if ans_list else ""
            for ans_list in examples["answers"]
        ]
        labels = tokenizer(
            targets, max_length=128, truncation=True, padding="max_length"
        )["input_ids"]
        model_inputs["labels"] = [
            [(lbl if lbl != tokenizer.pad_token_id else -100) for lbl in lab_seq]
            for lab_seq in labels
        ]
        return model_inputs

    train_ds = raw_train.map(preprocess_fn, batched=True, remove_columns=raw_train.column_names)
    dev_ds   = raw_dev.map(preprocess_fn,   batched=True, remove_columns=raw_dev.column_names)

    # 4. Data collator
    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

    # 5. Training args
    training_args = TrainingArguments(
        output_dir=save_model_path,
        eval_strategy="steps",
        eval_steps=500,
        save_steps=1000,
        save_total_limit=3,
        logging_steps=200,
        learning_rate=3e-5,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=8,
        num_train_epochs=num_epochs,
        warmup_steps=500,
        weight_decay=0.01,
        fp16=True,
    )

    # 6. Validation metric
    metric = evaluate.load("squad_v2")
    val_refs = [
        {
            "id": ex["id"],
            "answers": {
                "text": [a["text"] for a in ex["answers"]],
                "answer_start": [a["answer_start"] for a in ex["answers"]],
            }
        }
        for ex in raw_dev
    ]
    def compute_metrics(eval_pred):
        preds, _ = eval_pred
        decoded = tokenizer.batch_decode(preds, skip_special_tokens=True)
        formatted = [
            {"id": id_, "prediction_text": pred.strip()}
            for id_, pred in zip(raw_dev["id"], decoded)
        ]
        res = metric.compute(predictions=formatted, references=val_refs)
        return {"exact_match": res["exact_match"], "f1": res["f1"]}

    # 7. Trainer & train
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_ds,
        eval_dataset=dev_ds,
        data_collator=data_collator,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )
    trainer.train()
    trainer.save_model(save_model_path)
    tokenizer.save_pretrained(save_model_path)
    print(f"Model and tokenizer saved to {save_model_path}")

    return trainer


def evaluate_model(
    model_path: str,
    test_file: str,
    prefix: str = "question: ",
    max_gen_length: int = 128,
    num_beams: int = 4,
):
    # 1. Load model & tokenizer
    model     = T5ForConditionalGeneration.from_pretrained(model_path)
    tokenizer = T5TokenizerFast.from_pretrained(model_path)
    model.eval()
    device = "cuda" if model.device.type == "cuda" else "cpu"

    # 2. Raw test examples and SQuAD refs
    raw_test_list = load_radqa(test_file)
    test_refs = [
        {
            "id": ex["id"],
            "answers": {
                "text": [a["text"] for a in ex["answers"]],
                "answer_start": [a["answer_start"] for a in ex["answers"]],
            }
        }
        for ex in raw_test_list
    ]

    # 3. Build 🤗 Dataset & tokenize (no labels needed)
    ds = Dataset.from_list(raw_test_list)
    def preprocess(ex):
        inputs = prefix + ex["question"] + "  context: " + ex["context"]
        enc = tokenizer(
            inputs, max_length=512, truncation=True, padding="max_length", return_tensors="pt"
        )
        return {"input_ids": enc.input_ids[0], "attention_mask": enc.attention_mask[0]}
    test_ds = ds.map(preprocess, remove_columns=ds.column_names)

    # 4. Data collator & Trainer (for dataloader)
    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
    args = TrainingArguments(
        output_dir="eval_tmp",
        per_device_eval_batch_size=8,
        do_train=False,
        do_eval=False,
        logging_dir=None,
        report_to="none",          # silences wandb warning
    )
    trainer = Trainer(
        model=model,
        args=args,
        tokenizer=tokenizer,
        data_collator=data_collator,
    )
    test_loader = trainer.get_eval_dataloader(test_ds)

    # 5. Generation loop
    all_preds = []
    for batch in test_loader:
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        generated = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_length=max_gen_length,
            num_beams=num_beams,
        )
        decoded = tokenizer.batch_decode(generated, skip_special_tokens=True)
        all_preds.extend([d.strip() for d in decoded])

 # 6. Compute SQuAD v2 metrics (include no_answer_probability)
    formatted_preds = []
    for ex, pred in zip(raw_test_list, all_preds):
        p = pred.strip()
        # if model returned empty string, we signal "no answer" with prob=1.0
        no_ans_prob = 1.0 if p == "" else 0.0
        formatted_preds.append({
            "id": ex["id"],
            "prediction_text": p,
            "no_answer_probability": no_ans_prob,
        })

    metric = evaluate.load("squad_v2")
    results = metric.compute(predictions=formatted_preds, references=test_refs)

    return results


## 2.2 Fintune mimic4 pretrained T5 base on RadQA

In [4]:
# Train a model based on T5 pretrained on MIMIC
trainer = train_model(
      base_model_path="/content/drive/MyDrive/uiuc/t5_base_mimic",
      save_model_path="/content/drive/MyDrive/uiuc/radqa_finetuned_t5_base_mimic",
      train_file="/content/drive/MyDrive/uiuc/radQA/train.json",
      dev_file="/content/drive/MyDrive/uiuc/radQA/dev.json",
      num_epochs=3,
      n=500,
  )

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/656 [00:00<?, ? examples/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-3-7feb00faf83c>:125: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss


Model and tokenizer saved to /content/drive/MyDrive/uiuc/radqa_finetuned_t5_base_mimic


In [5]:
# Evaluation on test split for model trained on mimic pretrained T5
metrics = evaluate_model(
    model_path="/content/drive/MyDrive/uiuc/radqa_finetuned_t5_base_mimic",
    test_file="/content/drive/MyDrive/uiuc/radQA/test.json",
)

Map:   0%|          | 0/614 [00:00<?, ? examples/s]

<ipython-input-3-7feb00faf83c>:188: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [6]:
metrics

{'exact': 9.77198697068404,
 'f1': 26.435642814820802,
 'total': 614,
 'HasAns_exact': 2.8260869565217392,
 'HasAns_f1': 25.068444974565157,
 'HasAns_total': 460,
 'NoAns_exact': 30.51948051948052,
 'NoAns_f1': 30.51948051948052,
 'NoAns_total': 154,
 'best_exact': 25.570032573289904,
 'best_exact_thresh': 0.0,
 'best_f1': 26.598509264332197,
 'best_f1_thresh': 0.0}

## 2.3 Directly finetune on T5 base

In [27]:
# Train a model based on T5 base - NO Pretraining
train_model(
    base_model_path="t5-base",
    save_model_path="/content/drive/MyDrive/uiuc/radqa_finetuned_t5_base",
    train_file="/content/drive/MyDrive/uiuc/radQA/train.json",
    dev_file="/content/drive/MyDrive/uiuc/radQA/dev.json",
    num_epochs=3,
    n=500,
)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/656 [00:00<?, ? examples/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-16-8aaceda30002>:125: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss,Validation Loss


Model and tokenizer saved to /content/drive/MyDrive/uiuc/radqa_finetuned_t5_base


In [28]:
# Evaluation on test split for model trained on mimic pretrained T5
metrics = evaluate_model(
    model_path="/content/drive/MyDrive/uiuc/radqa_finetuned_t5_base",
    test_file="/content/drive/MyDrive/uiuc/radQA/test.json",
)

Map:   0%|          | 0/614 [00:00<?, ? examples/s]

<ipython-input-16-8aaceda30002>:188: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [29]:
metrics

{'exact': 29.80456026058632,
 'f1': 52.38366284656477,
 'total': 614,
 'HasAns_exact': 29.782608695652176,
 'HasAns_f1': 59.92080214737122,
 'HasAns_total': 460,
 'NoAns_exact': 29.87012987012987,
 'NoAns_f1': 29.87012987012987,
 'NoAns_total': 154,
 'best_exact': 29.96742671009772,
 'best_exact_thresh': 0.0,
 'best_f1': 52.54652929607615,
 'best_f1_thresh': 0.0}

## 2.4 Directly finetune on Clinical-T5-Base-Ckpt

In [13]:
# Train a model based on T5 base - NO Pretraining
train_model(
    base_model_path="hossboll/clinical-t5",
    save_model_path="/content/drive/MyDrive/uiuc/radqa_finetuned_clinical_t5_base",
    train_file="/content/drive/MyDrive/uiuc/radQA/train.json",
    dev_file="/content/drive/MyDrive/uiuc/radQA/dev.json",
    num_epochs=3,
    n=500,
)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/656 [00:00<?, ? examples/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-11-7feb00faf83c>:125: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss,Validation Loss


Model and tokenizer saved to /content/drive/MyDrive/uiuc/radqa_finetuned_clinical_t5_base


In [14]:
# Evaluation on test split for model trained on mimic pretrained T5
metrics = evaluate_model(
    model_path="/content/drive/MyDrive/uiuc/radqa_finetuned_clinical_t5_base",
    test_file="/content/drive/MyDrive/uiuc/radQA/test.json",
)

Map:   0%|          | 0/614 [00:00<?, ? examples/s]

<ipython-input-11-7feb00faf83c>:188: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [15]:
metrics

{'exact': 19.54397394136808,
 'f1': 41.38319335034065,
 'total': 614,
 'HasAns_exact': 17.17391304347826,
 'HasAns_f1': 46.32452329806337,
 'HasAns_total': 460,
 'NoAns_exact': 26.623376623376622,
 'NoAns_f1': 26.623376623376622,
 'NoAns_total': 154,
 'best_exact': 25.407166123778502,
 'best_exact_thresh': 0.0,
 'best_f1': 41.546059799852095,
 'best_f1_thresh': 0.0}

2.6 Directly use T5-XL to predict

In [24]:
# Evaluation on test split for model trained on mimic pretrained T5
metrics = evaluate_model(
    model_path="google/flan-t5-xl",
    test_file="/content/drive/MyDrive/uiuc/radQA/test.json",
)

config.json:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.45G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Map:   0%|          | 0/614 [00:00<?, ? examples/s]

<ipython-input-16-8aaceda30002>:188: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [25]:
metrics

{'exact': 4.88599348534202,
 'f1': 22.031462517251487,
 'total': 614,
 'HasAns_exact': 6.521739130434782,
 'HasAns_f1': 29.40721301215742,
 'HasAns_total': 460,
 'NoAns_exact': 0.0,
 'NoAns_f1': 0.0,
 'NoAns_total': 154,
 'best_exact': 25.407166123778502,
 'best_exact_thresh': 0.0,
 'best_f1': 26.338370275914524,
 'best_f1_thresh': 0.0}